# Basic solutions

## Introduction to optimization and operations research

Michel Bierlaire


In [ ]:

from itertools import combinations
from typing import Iterable

import numpy as np
from teaching_optimization.linear_constraints import (
    draw_polyhedron_standard_form,
    LabeledPoint,
    BoundingBox,
    StandardForm,
)


The objective of this exercise is to associate the choice of basic and non basic variables with a vertex of
the polyhedron. We will proceed step by step.

We will implement the following functions:

- a function that extracts and returns a basic matrix from the given constraint matrix,
- a function that calculates the basic variables by solving the system $B x_B = b$,
- a function that checks if the basic variables $x_B$ are feasible,
- a function that builds the vector in the full space,
- a function that calls all the previous ones to obtain a basic solution,
- a function that checks if the basic solution is degenerate.

We will apply it to identify the vertices of the following polyhedrons.

## Polyhedron 1
$$P = \left\{
\begin{pmatrix}
x_1\\
x_2
\end{pmatrix} \in \mathbb{R}^2
| x_1+x_2  \geq 1,
x_1+x_2  \leq 2,
x_1  \geq 0,
x_2  \geq 0. \right\}.$$

## Polyhedron 2
$$P = \left\{
\begin{pmatrix}
x_1\\
x_2
\end{pmatrix}\in \mathbb{R}^2
| x_1 + x_2 \leq 1, 3 x_1 + 10 x_2 \leq 15, x_1 \geq 0, x_2 \geq 0 \right\}.$$

## Polyhedron 3
$$ P = \left\{
\begin{pmatrix}
x_1\\
x_2
\end{pmatrix} \in \mathbb{R}^2
| x_1+x_2 \leq 1,
-x_1+2x_2 \leq 2, x_1 \geq 0, x_2 \geq 0 \right\}.$$

Prepare first polyhedron 1 so that we can use it for testing each function. It must first be written in standard form.

The polyhedron can be written in standard form by introducing non
negative slack variables, and rewriting the constraints in the form of equality constraints:

\begin{align*}
-x_1-x_2+x_3 & =-1,\\
x_1+x_2+x_4 &=2,\\
x_1,x_2,x_3,x_4 & \geq 0.
\end{align*}

The equality constraints in matrix form are
$$
Ax= b,
\text{ where }
A \in \mathbb{R}^{2\times 4},\; b \in \mathbb{R}^2,\; x \in \mathbb{R}^4,
$$
$$
A =
\begin{pmatrix*}[r]
-1&-1&1&0\\
1&1&0&1
\end{pmatrix*},\;
b =
\begin{pmatrix*}[r]
-1\\
2
\end{pmatrix*}.
$$

Provide the values of the matrix of the problem in standard form.

In [ ]:
standard_a = np.array([[-1, -1, 1, 0], [1, 1, 0, 1]])
print(standard_a)


In [ ]:
n_constraints, n_variables = standard_a.shape
print(f'{n_variables} variables, {n_constraints} constraints.')


Provide the values of the right-hand side of the problem in standard form.

In [ ]:
standard_b = np.array([-1, 2])
print(standard_b)


Draw the polyhedron.

In [ ]:
draw_polyhedron_standard_form(matrix_a=standard_a, vector_b=standard_b)



## Function that extracts and returns a basic matrix from the given constraint matrix
Complete it by replacing the ????.

In [ ]:
def extract_basic_matrix(
    constraint_matrix: np.ndarray, basic_indices: list[int]
) -> np.ndarray:
    """
    Extracts and returns a basic matrix from the given constraint matrix
    using the specified basic indices.

    :param constraint_matrix: A numpy ndarray representing the constraint matrix.
    :param basic_indices: A list of integers representing the column indices to form the basis.
    :return: A numpy ndarray representing the basic matrix.
    """
    n_rows, n_columns = constraint_matrix.shape
    # Verify that the number of indices matches the number of rows in the matrix.
    if len(basic_indices) != n_rows:
        raise ValueError(
            f'The number of basic indices [{len(basic_indices)}] must match the number of '
            f'rows in the matrix [{n_rows}].'
        )

    # Verify each index is a valid column index of the matrix.
    max_index = n_columns - 1
    if not all(0 <= index <= max_index for index in basic_indices):
        raise ValueError(
            'One or more indices are out of the valid column index range of the matrix.'
        )

    # Extract the columns corresponding to the indices, in the provided order.
    basis = constraint_matrix[:, basic_indices]

    # Return the resulting square matrix.
    return basis



Test the function on the example. Remember that, in Python, the numbering starts at 0.

In [ ]:
print(standard_a)

basic_indices_1 = [0, 1]

Expected result: $$\begin{pmatrix} -1 & -1 \\ 1 & 1 \end{pmatrix}.$$

In [ ]:

basic_matrix_1 = extract_basic_matrix(
    constraint_matrix=standard_a, basic_indices=basic_indices_1
)
print(basic_matrix_1)


Test the function with a different set of indices. Note the order...

In [ ]:
basic_indices_2 = [3, 2]

Expected result: $$\begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}.$$

In [ ]:

basic_matrix_2 = extract_basic_matrix(
    constraint_matrix=standard_a, basic_indices=basic_indices_2
)
print(basic_matrix_2)



## Function that calculates the basic variables by solving the system $B x_B = b$.
Complete it by replacing the ????.

In [ ]:
def calculate_basic_variables(
    basic_matrix: np.ndarray, right_hand_side: np.ndarray
) -> np.ndarray | None:
    """
    Solves the system $B x_B = b$.

    :param basic_matrix: matrix $B$.
    :param right_hand_side: vector $b$.
    :return: if the system has a solution, returns $x_B$. If not, returns None.
    """
    # Check if the basic matrix is square
    n_rows, n_columns = basic_matrix.shape
    if n_rows != n_columns:
        raise ValueError(
            f'The basic matrix must be square, and not {n_rows} x {n_columns}.'
        )

    # Check if the length of the right-hand side matches the dimensions of the matrix
    if n_rows != len(right_hand_side):
        raise ValueError(
            f'The length of the right-hand side [{len(right_hand_side)}] must match '
            f'the dimensions of the matrix [{n_rows}].'
        )

    # Solve the system B x_B = b, if possible, using `np.linalg.solve`.

    try:
        basic_variables = np.linalg.solve(basic_matrix, right_hand_side)
        return basic_variables
    except np.linalg.LinAlgError:
        # The system does not have a solution, as the matrix is not invertible.
        return None



Test the function. Expected result: None.

In [ ]:
basic_variables_1 = calculate_basic_variables(
    basic_matrix=basic_matrix_1, right_hand_side=standard_b
)
print(basic_variables_1)


The system indeed has no solution:

In [ ]:
try:
    basic_variables_1 = np.linalg.solve(basic_matrix_1, standard_b)
except np.linalg.LinAlgError as error:
    print(error)

In [ ]:
print(basic_matrix_1)


Test again the function. Expected result: $$\left(\begin{array}{c}2 \\ -1 \end{array}\right).$$

In [ ]:
basic_variables_2 = calculate_basic_variables(
    basic_matrix=basic_matrix_2, right_hand_side=standard_b
)
print(basic_variables_2)



## Function that checks if the basic variables $x_B$ are feasible.
Complete it by replacing the ????.

In [ ]:
def check_feasibility(basic_variables: np.ndarray) -> bool:
    """
    Check if the basic variables $x_B$ are feasible.

    :param basic_variables: basic variables $x_B$.
    :return: True if feasible. False otherwise,
    """
    is_feasible: bool = np.all(basic_variables >= 0)
    return is_feasible



Test the function. Expected result: False

In [ ]:
is_basis_2_feasible = check_feasibility(basic_variables=basic_variables_2)
print(is_basis_2_feasible)



## Function that checks if the basic variables $x_B$ are degenerate.
Complete it by replacing the ????.

In [ ]:
def check_degeneracy(basic_variables: np.ndarray) -> bool:
    """
    Check if the basic variables $x_B$ are degenerate.

    :param basic_variables: basic variables $x_B$.
    :return: True if degenerate. False otherwise,
    """
    is_degenerate: bool = np.any(
        basic_variables == 0
    )
    return is_degenerate



Test the function. Expected result: False

In [ ]:
is_basis_2_degenerate = check_degeneracy(basic_variables=basic_variables_2)
print(is_basis_2_degenerate)



## Function that builds the vector in the full space.
In a space of dimension $n$, we have a vector $x_B$ of basic variables of dimension $m \leq n$, identified
by their indices. This function builds the vector in dimension $n$ where all non basic variables are zero.
Complete it by replacing the ????.

In [ ]:
def build_solution(
    number_of_variables: int, basic_variables: np.ndarray, basic_indices: Iterable[int]
) -> np.ndarray:
    """
    Builds the vector in dimension n where all non basic variables are zero.

    :param number_of_variables: dimension n.
    :param basic_variables: vector of basic variables.
    :param basic_indices: indices of basic variables in the space of dimension n.
    :return: complete solution.
    """
    # Check that the number of basic variables is less or equal to n
    basic_indices = list(basic_indices)
    if len(basic_variables) > number_of_variables:
        raise ValueError(
            f'The number of basic variables [{len(basic_variables)}] must be less than or equal '
            f'to the dimension [{number_of_variables}].'
        )

    # Check that the two vectors have the same dimension
    if len(basic_variables) != len(basic_indices):
        raise ValueError(
            f'The dimensions of the basic variables [{len(basic_variables)}] and their '
            f'indices [{len(basic_indices)}] must match.'
        )

    # Check that all indices are valid
    if not all(0 <= index < number_of_variables for index in basic_indices):
        raise ValueError('All indices must be valid, that is between 0 and n-1.')

    # Initialize the complete solution vector with zeros
    complete_solution = np.zeros(number_of_variables)

    # Assign the values of basic variables to their corresponding positions
    complete_solution[basic_indices] = basic_variables

    return complete_solution



Test the function. Expected result: $$\left(\begin{array}{c}0 \\ 0  \\ -1 \\ 2 \end{array}\right).$$

In [ ]:

solution_2 = build_solution(
    number_of_variables=n_variables,
    basic_variables=basic_variables_2,
    basic_indices=basic_indices_2,
)
print(solution_2)



## Function that calls all the previous ones to obtain a basic solution.
Nothing to be completed.

In [ ]:
def calculate_basic_solution(
    constraint_matrix: np.ndarray, right_hand_side: np.ndarray, basic_indices: list[int]
) -> tuple[np.ndarray | None, bool, bool]:
    """
    Consider a polyhedron in standard form characterized by the $m \times n$ matrix $A$ and the vector $b$.
    Given the list of indices of basic variables, calculate the basic solution in standard form, and check feasibility.

    :param constraint_matrix: matrix $A$.
    :param right_hand_side: vector $b$.
    :param basic_indices: list of indices of basic variables.
    :return: the basic vector (or None if it does not exist),  its feasibility status, and its degeneracy status.
    """
    n_rows, n_columns = constraint_matrix.shape

    # Check the matrix is m x n such that n >= m
    if n_columns < n_rows:
        raise ValueError(
            f'Validation failed: The number of columns [{n_columns}] must be greater than or '
            f'equal to the number of rows [{n_rows}].'
        )

    # Check the vector has m elements
    if len(right_hand_side) != n_rows:
        raise ValueError(
            f'The rhs must have the same number of elements as there are rows in the '
            f'matrix [{len(right_hand_side)} != {n_rows}].'
        )

    # Check the list of indices
    if len(basic_indices) != n_rows:
        raise ValueError(
            f'The list of indices must be of the same length as there are rows in the '
            f'matrix [{len(basic_indices)} != {n_rows}].'
        )

    # Check all indices are between 0 and n-1
    if not all(0 <= index < n_columns for index in basic_indices):
        raise ValueError(
            f'Validation failed: All indices must be between 0 and {n_columns-1}.'
        )

    # Extract the basic matrix
    the_basic_matrix: np.ndarray = extract_basic_matrix(
        constraint_matrix=constraint_matrix, basic_indices=basic_indices
    )

    # Calculate the basic variables
    the_basic_variables: np.ndarray | None = calculate_basic_variables(
        basic_matrix=the_basic_matrix, right_hand_side=right_hand_side
    )

    # Check if the basic solution exists.
    if the_basic_variables is None:
        # The is no basic solution.
        return None, False, False

    feasibility: bool = check_feasibility(basic_variables=the_basic_variables)
    degeneracy: bool = check_degeneracy(basic_variables=the_basic_variables)

    the_basic_solution: np.ndarray = build_solution(
        number_of_variables=n_columns,
        basic_variables=the_basic_variables,
        basic_indices=basic_indices,
    )

    return the_basic_solution, feasibility, degeneracy



We test the function on all possible bases.

We need to consider all combinations of basic variables.

In [ ]:
all_bases: list[int] = list(combinations(range(n_variables), n_constraints))
print(f'There are {len(all_bases)} potential bases.')



To avoid repeating the code, we define a function that performs the analysis and the reporting for a given
list of indices.

In [ ]:
def print_info(basic_indices: Iterable[int] | int) -> np.ndarray:
    """Print the information related to the list of indices.

    :param basic_indices: list of basic indices.
    :return: the basic solution
    """
    basic_solution, feasible, degenerate = calculate_basic_solution(
        constraint_matrix=standard_a,
        right_hand_side=standard_b,
        basic_indices=list(basic_indices),
    )
    print(f' *** Basic indices: {basic_indices}')
    if basic_solution is None:
        print('No basic solution')
    else:
        feasible_text = '[feasible]' if feasible else '[non feasible]'
        degenerate_text = '[degenerate]' if degenerate else '[not degenerate]'
        print(f'Basic solution: {basic_solution} {feasible_text} {degenerate_text}')
    return basic_solution



## For each potential base below, verify that the outcome of the function is correct.

### Potential base number 0

In [ ]:
basic_indices_0 = all_bases[0]
basic_solution_0 = print_info(basic_indices_0)


Plot the vertex

In [ ]:
if basic_solution_0 is not None:
    draw_polyhedron_standard_form(
        matrix_a=standard_a,
        vector_b=standard_b,
        points=[LabeledPoint(coordinates=basic_solution_0)],
    )


### Potential base number 1

In [ ]:
basic_indices_1 = all_bases[1]
basic_solution_1 = print_info(basic_indices_1)


Plot the vertex

In [ ]:
if basic_solution_1 is not None:
    draw_polyhedron_standard_form(
        matrix_a=standard_a,
        vector_b=standard_b,
        points=[LabeledPoint(coordinates=basic_solution_1)],
    )


### Potential base number 2

In [ ]:
basic_indices_2 = all_bases[2]
basic_solution_2 = print_info(basic_indices_2)


Plot the vertex

In [ ]:
if basic_solution_2 is not None:
    draw_polyhedron_standard_form(
        matrix_a=standard_a,
        vector_b=standard_b,
        points=[LabeledPoint(coordinates=basic_solution_2)],
    )

### Potential base number 3

In [ ]:
basic_indices_3 = all_bases[3]
basic_solution_3 = print_info(basic_indices_3)


Plot the vertex

In [ ]:
if basic_solution_3 is not None:
    draw_polyhedron_standard_form(
        matrix_a=standard_a,
        vector_b=standard_b,
        points=[LabeledPoint(coordinates=basic_solution_3)],
    )

### Potential base number 4

In [ ]:
basic_indices_4 = all_bases[4]
basic_solution_4 = print_info(basic_indices_4)


Plot the vertex

In [ ]:
if basic_solution_4 is not None:
    draw_polyhedron_standard_form(
        matrix_a=standard_a,
        vector_b=standard_b,
        points=[LabeledPoint(coordinates=basic_solution_4)],
    )

### Potential base number 5

In [ ]:
basic_indices_5 = all_bases[5]
basic_solution_5 = print_info(basic_indices_5)


Plot the vertex

In [ ]:
if basic_solution_5 is not None:
    draw_polyhedron_standard_form(
        matrix_a=standard_a,
        vector_b=standard_b,
        points=[LabeledPoint(coordinates=basic_solution_5)],
    )


Among the 6 potential bases, only 5 exist: 4 feasible ane 1unfeasible. The 4 feasible basic solutions
correspond to the 4 vertices of the polyhedron.

## Polyhedron 2
$$P = \left\{
\begin{pmatrix}
x_1\\
x_2
\end{pmatrix}\in \mathbb{R}^2
| x_1 + x_2 \leq 1, 3 x_1 + 10 x_2 \leq 15, x_1 \geq 0, x_2 \geq 0 \right\}.$$

Data in standard form.

In [ ]:
standard_a = np.array([[1, 1, 1, 0], [3, 10, 0, 1]])
standard_b = np.array([1, 15])


In [ ]:
n_constraints, n_variables = standard_a.shape
print(f'{n_variables} variables, {n_constraints} constraints.')


Draw the polyhedron. We specify the bounding box in order to display all bases, even the unfeasible ones.

In [ ]:
bounding_box = BoundingBox(x_min=-1.5, x_max=5.5, y_min=-0.5, y_max=2)
draw_polyhedron_standard_form(
    matrix_a=standard_a, vector_b=standard_b, bounding_box=bounding_box
)

We test the function on all possible bases.

We need to consider all combinations of basic variables.

In [ ]:
all_bases = list(combinations(range(n_variables), n_constraints))
print(f'There are {len(all_bases)} potential bases.')


## For each potential base below, verify that the outcome of the function is correct.

### Potential base number 0

In [ ]:
basic_indices_0 = all_bases[0]
basic_solution_0 = print_info(basic_indices_0)


Plot the vertex

In [ ]:
if basic_solution_0 is not None:
    draw_polyhedron_standard_form(
        matrix_a=standard_a,
        vector_b=standard_b,
        bounding_box=bounding_box,
        points=[LabeledPoint(coordinates=basic_solution_0)],
    )


### Potential base number 1

In [ ]:
basic_indices_1 = all_bases[1]
basic_solution_1 = print_info(basic_indices_1)


Plot the vertex

In [ ]:
if basic_solution_1 is not None:
    draw_polyhedron_standard_form(
        matrix_a=standard_a,
        vector_b=standard_b,
        bounding_box=bounding_box,
        points=[LabeledPoint(coordinates=basic_solution_1)],
    )


### Potential base number 2

In [ ]:
basic_indices_2 = all_bases[2]
basic_solution_2 = print_info(basic_indices_2)


Plot the vertex

In [ ]:
if basic_solution_2 is not None:
    draw_polyhedron_standard_form(
        matrix_a=standard_a,
        vector_b=standard_b,
        bounding_box=bounding_box,
        points=[LabeledPoint(coordinates=basic_solution_2)],
    )

### Potential base number 3

In [ ]:
basic_indices_3 = all_bases[3]
basic_solution_3 = print_info(basic_indices_3)


Plot the vertex

In [ ]:
if basic_solution_3 is not None:
    draw_polyhedron_standard_form(
        matrix_a=standard_a,
        vector_b=standard_b,
        bounding_box=bounding_box,
        points=[LabeledPoint(coordinates=basic_solution_3)],
    )

### Potential base number 4

In [ ]:
basic_indices_4 = all_bases[4]
basic_solution_4 = print_info(basic_indices_4)


Plot the vertex

In [ ]:
if basic_solution_4 is not None:
    draw_polyhedron_standard_form(
        matrix_a=standard_a,
        vector_b=standard_b,
        bounding_box=bounding_box,
        points=[LabeledPoint(coordinates=basic_solution_4)],
    )

### Potential base number 5

In [ ]:
basic_indices_5 = all_bases[5]
basic_solution_5 = print_info(basic_indices_5)


Plot the vertex

In [ ]:
if basic_solution_5 is not None:
    draw_polyhedron_standard_form(
        matrix_a=standard_a,
        vector_b=standard_b,
        bounding_box=bounding_box,
        points=[LabeledPoint(coordinates=basic_solution_5)],
    )


All 6 potential bases exist. Only 3 are feasible They correspond to the 3 vertices of the polyhedron.

## Polyhedron 3
$$ P = \left\{
\begin{pmatrix}
x_1\\
x_2
\end{pmatrix} \in \mathbb{R}^2
| x_1+x_2 \leq 1,
-x_1+2x_2 \leq 2, x_1 \geq 0, x_2 \geq 0 \right\}.$$

Data in standard form.

In [ ]:
standard_a = np.array([[1, 1, 1, 0], [-1, 2, 0, 1]])
standard_b = np.array([1, 2])


In [ ]:
n_constraints, n_variables = standard_a.shape
print(f'{n_variables} variables, {n_constraints} constraints.')


Draw the polyhedron. We specify the bounding box in order to display all bases, even the unfeasible ones.

In [ ]:
bounding_box = BoundingBox(x_min=-2.5, x_max=1.5, y_min=-0.5, y_max=1.5)
draw_polyhedron_standard_form(
    matrix_a=standard_a, vector_b=standard_b, bounding_box=bounding_box
)

We test the function on all possible bases.

We need to consider all combinations of basic variables.

In [ ]:
all_bases = list(combinations(range(n_variables), n_constraints))
print(f'There are {len(all_bases)} potential bases.')


## For each potential base below, verify that the outcome of the function is correct.

### Potential base number 0

In [ ]:
basic_indices_0 = all_bases[0]
basic_solution_0 = print_info(basic_indices_0)


Plot the vertex

In [ ]:
if basic_solution_0 is not None:
    draw_polyhedron_standard_form(
        matrix_a=standard_a,
        vector_b=standard_b,
        bounding_box=bounding_box,
        points=[LabeledPoint(coordinates=basic_solution_0)],
    )


### Potential base number 1

In [ ]:
basic_indices_1 = all_bases[1]
basic_solution_1 = print_info(basic_indices_1)


Plot the vertex

In [ ]:
if basic_solution_1 is not None:
    draw_polyhedron_standard_form(
        matrix_a=standard_a,
        vector_b=standard_b,
        bounding_box=bounding_box,
        points=[LabeledPoint(coordinates=basic_solution_1)],
    )


### Potential base number 2

In [ ]:
basic_indices_2 = all_bases[2]
basic_solution_2 = print_info(basic_indices_2)


Plot the vertex

In [ ]:
if basic_solution_2 is not None:
    draw_polyhedron_standard_form(
        matrix_a=standard_a,
        vector_b=standard_b,
        bounding_box=bounding_box,
        points=[LabeledPoint(coordinates=basic_solution_2)],
    )

### Potential base number 3

In [ ]:
basic_indices_3 = all_bases[3]
basic_solution_3 = print_info(basic_indices_3)


Plot the vertex

In [ ]:
if basic_solution_3 is not None:
    draw_polyhedron_standard_form(
        matrix_a=standard_a,
        vector_b=standard_b,
        bounding_box=bounding_box,
        points=[LabeledPoint(coordinates=basic_solution_3)],
    )

### Potential base number 4

In [ ]:
basic_indices_4 = all_bases[4]
basic_solution_4 = print_info(basic_indices_4)


Plot the vertex

In [ ]:
if basic_solution_4 is not None:
    draw_polyhedron_standard_form(
        matrix_a=standard_a,
        vector_b=standard_b,
        bounding_box=bounding_box,
        points=[LabeledPoint(coordinates=basic_solution_4)],
    )

### Potential base number 5

In [ ]:
basic_indices_5 = all_bases[5]
basic_solution_5 = print_info(basic_indices_5)


Plot the vertex

In [ ]:
if basic_solution_5 is not None:
    draw_polyhedron_standard_form(
        matrix_a=standard_a,
        vector_b=standard_b,
        bounding_box=bounding_box,
        points=[LabeledPoint(coordinates=basic_solution_5)],
    )


All 6 potential bases exist, and 5 of them are feasible. However, the polyhedron has only 3 vertices.
Indeed, feasible basic solutions 0, 3 and 4 correspond to the exact same vertex of the
polyhedron. Such basic solutions are said to be **degenerate**.

# Future usage.
These functions are implemented in the package, so that they can be reused in the future. Here is the syntax.

First, we create an object using the data:

In [ ]:
the_standard_form = StandardForm(matrix=standard_a, vector=standard_b)


Then, we set the basic indices.

In [ ]:
the_basic_indices = [1, 2]
the_standard_form.basic_indices = the_basic_indices


The basic matrix is readily available.

In [ ]:
print(the_standard_form.basic_matrix)


As well as the basic solution:

In [ ]:
print(the_standard_form.basic_solution)


If you need only the basic part of the basic solution, it is available as well.

In [ ]:
print(the_standard_form.basic_part_basic_solution)


To change the basis, redefine the basic indices.

In [ ]:
the_basic_indices = [0, 3]
the_standard_form.basic_indices = the_basic_indices


The basic matrix is readily available.

In [ ]:
print(the_standard_form.basic_matrix)


As well as the basic solution:

In [ ]:
print(the_standard_form.basic_solution)


If you need only the basic part of the basic solution, it is available as well.

In [ ]:
print(the_standard_form.basic_part_basic_solution)
